In [1]:
from dvc.repo import Repo as DVCRepo
import pathlib
r = DVCRepo(r"D:\ig_pipeline")

In [2]:
# Boolean flag determining whether or not stages will be checked for status
CHECK_PROCESSED = True

In [3]:
stages = r.stages

In [4]:
dir(next(s for s in stages if s.is_data_source))
print(pathlib.Path(next(s for s in stages if s.is_data_source).outs[0].fs_path).name)

processed.max


In [5]:
s_to_deps = {s: [d.fs_path for d in s.deps] for s in stages}
out_to_s = {o.fs_path: s for s in stages for o in s.outs}

In [6]:
import collections
out_to_s_dup = collections.defaultdict(list)
for s in stages:
    for o in s.outs:
        out_to_s_dup[o.fs_path].append(s)

In [7]:
[x for x, v in out_to_s_dup.items() if len(v) > 1]

[]

In [8]:
import networkx as nx

In [9]:
G = nx.DiGraph()

In [10]:
def name(s):
    if s.is_data_source:
        return pathlib.Path(s.outs[0].fs_path).name
    return s.name

In [11]:
def canonicalize(s):
    if s.is_data_source:
        return pathlib.Path(s.outs[0].fs_path).name
        
    return s.name.split("@")[0] if not s.is_data_source else "data_source"

In [12]:
if CHECK_PROCESSED:
    r.lock.lock()

# Process nodes
for s in s_to_deps.keys():
    this_node = canonicalize(s)
    if this_node not in G.nodes:
        G.add_node(this_node, total=0, changed=0, total_set=set(), changed_set=set())
        
    if CHECK_PROCESSED:
        G.nodes[this_node]["total"] += 1
        G.nodes[this_node]["total_set"].add(name(s))
        if s.changed():
            G.nodes[this_node]["changed"] += 1
            G.nodes[this_node]["changed_set"].add(name(s))
    
# Add dependencies
for s in s_to_deps.keys():
    this_node = canonicalize(s)
    for dep in s_to_deps[s]:
        if dep in out_to_s:
            from_node = canonicalize(out_to_s[dep])
            G.add_edge(from_node, this_node)
            
if CHECK_PROCESSED:
    r.lock.unlock()

In [13]:
for x in nx.topological_sort(G):
    print(x)

processed.max
raw.max
textures
collision_meshes.zip
bathroom.max
living.max
proxy
object_list
room_object_list
sanitycheck
object_list_success
combined_room_object_list
combined_room_object_list_future
sanitycheck_success
object_inventory
object_inventory_future
export_meshes
aggregate_metadata
object_inventory_success
export_meshes_success
aggregate_metadata_success
export_objs_global
export_scene
export_objs_global_success
usdify_objects
aggregate_scenes
generate_object_images
make_maps
pack_dataset
validate_scene
usdify_dataset


In [14]:
# In mermaid format for pasting into README
for f, t in G.edges:
    print(f"    {f} --> {t}")

    aggregate_metadata --> aggregate_metadata_success
    aggregate_metadata --> make_maps
    aggregate_metadata --> pack_dataset
    aggregate_metadata --> usdify_objects
    aggregate_metadata_success --> pack_dataset
    aggregate_metadata_success --> usdify_objects
    aggregate_scenes --> make_maps
    aggregate_scenes --> pack_dataset
    aggregate_scenes --> validate_scene
    export_meshes --> export_meshes_success
    export_meshes --> export_objs_global
    export_meshes --> export_scene
    export_meshes_success --> export_objs_global
    export_meshes_success --> export_scene
    export_objs_global --> export_objs_global_success
    export_objs_global --> make_maps
    export_objs_global --> pack_dataset
    export_objs_global --> usdify_objects
    export_objs_global_success --> pack_dataset
    export_scene --> aggregate_scenes
    object_inventory --> aggregate_metadata
    object_inventory --> object_inventory_success
    object_list --> export_meshes
    object_list -

In [15]:
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
pos = graphviz_layout(G, prog="dot")
nx.draw(G, pos)
plt.show()

"dot" with args ['-Tdot', 'C:\\Users\\Cem\\AppData\\Local\\Temp\\tmpattw4tkr'] returned code: 1

stdout, stderr:
 b''
b'Warning: Could not load "C:\\Users\\Cem\\.conda\\envs\\pipeline\\Library\\bin\\gvplugin_pango.dll" - It was found, so perhaps one of its dependents was not.  Try ldd.\r\nError: C:\\Users\\Cem\\AppData\\Local\\Temp\\tmpattw4tkr: syntax error in line 7 scanning a quoted string (missing endquote? longer than 16384?)\r\nString starting:"{\'export_meshes@objects/legacy_shelf-njwsoa\', \'export_meshes@objects/legacy_tart\r\n'



AssertionError: "dot" with args ['-Tdot', 'C:\\Users\\Cem\\AppData\\Local\\Temp\\tmpattw4tkr'] returned code: 1

In [16]:
if CHECK_PROCESSED:
    print("Completion ratios:")
    for x in nx.topological_sort(G):
        completion_ratio = 1 - (G.nodes[x]["changed"] / G.nodes[x]["total"])
        completion_percentage = int(completion_ratio * 100)
        print(f"{x}: {completion_percentage}%")

Completion ratios:
processed.max: 100%
raw.max: 100%
textures: 100%
collision_meshes.zip: 100%
bathroom.max: 100%
living.max: 100%
proxy: 100%
object_list: 100%
room_object_list: 100%
sanitycheck: 100%
object_list_success: 0%
combined_room_object_list: 0%
combined_room_object_list_future: 0%
sanitycheck_success: 100%
object_inventory: 0%
object_inventory_future: 0%
export_meshes: 18%
aggregate_metadata: 0%
object_inventory_success: 0%
export_meshes_success: 0%
aggregate_metadata_success: 0%
export_objs_global: 0%
export_scene: 0%
export_objs_global_success: 0%
usdify_objects: 0%
aggregate_scenes: 0%
generate_object_images: 0%
make_maps: 0%
pack_dataset: 0%
validate_scene: 0%
usdify_dataset: 0%


In [20]:
completed_meshes = G.nodes["export_meshes"]["total_set"] - G.nodes["export_meshes"]["changed_set"]
for e in sorted(completed_meshes):
    obj = e.split("/")[-1]
    print(f'    "{obj}",')


    "acetone-jc",
    "adhesive-io",
    "air_filter-ls",
    "alfredo_sauce-zq",
    "antlers-mq",
    "arepa-od",
    "asparagus-ew",
    "axe-yy",
    "bar_cart-gz",
    "basball_bat-gx",
    "beeswax-ad",
    "birdseed-bd",
    "bread-nx",
    "brussel_sprouts-zf",
    "cake-fg",
    "candy-mb",
    "canvas-ts",
    "catalog-xj",
    "cell_phone-nb",
    "ceramic_tile-yr",
    "chalkboard-cl",
    "champagne-cs",
    "champagne_flute-pb",
    "cheesecake-tc",
    "chili_pepper-ad",
    "citrus-ln",
    "coke_bottle-al",
    "comic_book-lr",
    "copper_wire-rh",
    "corn-ds",
    "cotton_balls-ps",
    "cracker-fg",
    "creme_brulee-eu",
    "cushion-kh",
    "deodorant-ls",
    "doll-hu",
    "dried_cranberries-wt",
    "dumpling-uy",
    "dutch_oven-wa",
    "electric_kettle-la",
    "eyeglasses-op",
    "fabric_softener-od",
    "fan-js",
    "fertilizer_bag-oa",
    "fire_wood-mr",
    "fireplace-pf",
    "fishing_rod-gz",
    "food_processor-ht",
    "french_fries-jh",
    "

    "mixing_bowl-fp",
    "moth_ball-pg",
    "omelet-06",
    "onion_powder-a9",
    "pancake-7b",
    "parsnip-7h",
    "pastry_cutter-bx",
    "pea_pods-hg",
    "peanut_butter-pa",
    "pet_bed-yw",
    "pillowcase-ye",
    "pin-gl",
    "plush_toy-de",
    "postage_stamps-fn",
    "power_strip-le",
    "pressure_cooker-7c",
    "purse-go",
    "quail_breast_raw-5i",
    "radio-jn",
    "receipt-hu",
    "rice_cooker-av",
    "risotto-uj",
    "ruler-es",
    "sandwich-uw",
    "seashell-pe",
    "shirt-pe",
    "shoe-mn",
    "shovel-xa",
    "sneakers-cr",
    "soccer_ball-uj",
    "softball-jr",
    "spices-pa",
    "spray_bottle-ea",
    "sriracha-bb",
    "stapler-tr",
    "steel_wool",
    "succulent-kg",
    "sweatshirt-kd",
    "sweet_almond_oil-me",
    "switch-ou",
    "tackle_box-vu",
    "tennis_ball-zc",
    "thread-zl",
    "toy_train-vj",
    "truck-sf",
    "tuna-wh",
    "utility_knife-ba",
    "vanilla-yz",
    "vitamin-xs",
    "vodka-ht",
    "waffle-yd",
    "w